In [ ]:
!pip install sentencepiece
!pip install python-Levenshtein

In [ ]:
## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
!wget https://www.openslr.org/resources/12/train-clean-100.tar.gz

In [ ]:
!wget https://www.openslr.org/resources/12/test-clean.tar.gz

In [ ]:
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz

In [ ]:
!tar xvzf train-clean-100.tar.gz

In [ ]:
!tar xvzf dev-clean.tar.gz

In [ ]:
!tar xvzf test-clean.tar.gz

In [29]:
import torch
from torch import nn
import numpy as np
from torchaudio.models import RNNT,Emformer,emformer_rnnt_base,emformer_rnnt_model
from torch import optim
import torch
from torch import nn
from torchaudio import functional as audioF
from torchaudio import transforms as audioT
import torchaudio
import Levenshtein as Lev

In [30]:
import numpy as np
import torch
from torch import nn
from glob import glob
#import librosa
from tqdm import tqdm
#import soundfile as sf
import os
import random
from pathlib import Path

from parasol import Composer

In [31]:
def wer(s1,s2):
    b = set(s1.split()+s2.split())
    word_map = dict(zip(b,range(len(b))))
    print("word_map:",word_map)
    w1 = [chr(word_map[w]) for w in s1.split()]
    w2 = [chr(word_map[w]) for w in s2.split()]
    dev = max(len(s2.split()),1)
    score = Lev.distance("".join(w1),"".join(w2))/dev
    return score

def cer(s1,s2):
    w1 = s1.replace(" ","")
    w2 = s2.replace(" ","")
    dev = max(len(s2),1)
    score = Lev.distance((w1),(w2))/dev
    return score

In [32]:

chosung = ("ㄱ", "ㄲ", "ㄴ", "ㄷ", "ㄸ", "ㄹ", "ㅁ", "ㅂ", "ㅃ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅉ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

jungsung = ("ㅏ", "ㅐ", "ㅑ", "ㅒ", "ㅓ", "ㅔ", "ㅕ", "ㅖ", "ㅗ", "ㅘ", "ㅙ", "ㅚ", "ㅛ", "ㅜ", "ㅝ", "ㅞ", "ㅟ", "ㅠ", "ㅡ", "ㅢ", "ㅣ")

jongsung = ("", "ㄱ", "ㄲ", "ㄳ", "ㄴ", "ㄵ", "ㄶ", "ㄷ", "ㄹ", "ㄺ", "ㄻ", "ㄼ", "ㄽ", "ㄾ", "ㄿ", "ㅀ", "ㅁ", "ㅂ", "ㅄ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

ENGS = tuple([chr(ord("A")+i) for i in range(26)])

special_chars = tuple(list(" #$!?@~^&*()[]'~.\"`_:"))

import re

def replaceBracket(sentence):
    rst = re.sub( "(\(((\S|\s)*)(\)_))","",sentence)
    rst = re.sub( "(\(((\S|\s)*)(\)))","",rst)
    return rst

def getHangeulIndex(one_character):
    
    for i,comp in enumerate(chosung+jungsung+jongsung+ENGS+special_chars):
        
        if(one_character.upper()==comp):
            result = i
            if(i<len(chosung)):
                result = i
            elif(i<len(chosung)+len(jungsung)):
                result = i+len(chosung)
            elif(i<len(chosung)+len(jungsung)+len(jongsung)):
                result = i+len(chosung)+len(jungsung)
            return i+5
    return 4
def convertString2HangeulIndex(text):
    result = []
    for character in text:
        result.append(getHangeulIndex(character))
    return result

def isHangeul(one_character):
    return 0xAC00 <= ord(one_character[:1]) <= 0xD7A3

def hangeulExplode(one_hangeul):
    a = one_hangeul[:1]
    if isHangeul(a) != True:
        return False
    b = ord(a) - 0xAC00
    cho = b // (21*28)
    jung = b % (21*28) // 28
    jong = b % 28
    if jong == 0:
        return (chosung[cho], jungsung[jung],-1)
    else:
        return (chosung[cho], jungsung[jung], jongsung[jong],-1)

def hangeulJoin(inputlist):
    result = ""
    cho, jung, jong = 0, 0, 0
    inputlist.insert(0, "")
    while len(inputlist) > 1:
        if inputlist[-1] in jongsung:
            if inputlist[-2] in jungsung:
                jong = jongsung.index(inputlist.pop())
            
            else:
                result += inputlist.pop()
        elif inputlist[-1] in jungsung:
            if inputlist[-2] in chosung:
                jung = jungsung.index(inputlist.pop())
                cho = chosung.index(inputlist.pop())
                result += chr(0xAC00 + ((cho*21)+jung)*28+jong)
                cho, jung, jong = 0, 0, 0
            else:
                result += inputlist.pop()

        else:
            result += inputlist.pop()
    else:
        return result[::-1]

def pureosseugi(inputtext):
    result = ""
    for i in inputtext:
        if isHangeul(i) == True:
            for j in hangeulExplode(i):
                result += j
        else:
            result += i
    
    return result

def bracket_filter(sentence):
    new_sentence = str()
    flag = False
    
    for ch in sentence:
        if ch == '(' and flag == False: 
            flag = True
            continue
        if ch == '(' and flag == True:
            flag = False
            continue
        if ch != ')' and flag == False:
            new_sentence += ch
    return new_sentence

def special_filter(sentence):
    SENTENCE_MARK = ['?', '!']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', '.', ',']
    
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            # o/, n/ 등 처리
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
                continue 

        if ch == '#': 
            new_sentence += '샾'

        elif ch not in EXCEPT: 
            new_sentence += ch

    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence

def sentence_filter(raw_sentence):
    return special_filter(bracket_filter(raw_sentence))

def bracketContentFilter(text):
    #괄호가 페어를 이루고 있다는 전제로 짜여진 함수
    def contentExtractor(tmp_value):
        if(tmp_value.find(":")):
            tmp2 = tmp_value.split(":",1)
            return tmp2[-1]
        else:
            return tmp_value
    isBracket = False
    result = ""
    tmp = ""
    for c in text:
        if(c=="("):
            isBracket = True
        elif(c==")"):
            result += contentExtractor(tmp)
            isBracket = False
            tmp = ""
        elif(isBracket==True):
            tmp += c
        else:
            result += c
    return result

def text2vector(inputtext):
    def chosung2index(v):
        for i,cho in enumerate(chosung):
            if(v==cho):
                return i
        return -1
    def jungsung2index(v):
        for i,jung in enumerate(jungsung):
            if(jung==v):
                return i+len(chosung)
        return -1
    def jongsung2index(v):
        for i,jong in enumerate(jongsung):
            if(jong==v):
                return i+len(chosung)+len(jungsung)
        return -1
    chojongsung = list(set(chosung+jungsung))
    def chojongsung2index(v):
        for i,jong in enumerate(chojongsung):
            if(jong==v):
                return i+len(chosung)+len(jungsung)
        return -1
    
    result = []
    etc_chars = list(ENGS+special_chars)
    for i in inputtext:
        if isHangeul(i) == True:
            hexp = hangeulExplode(i)
            hexp_len = len(hexp)
            if(hexp_len==4):
                cho,jung,jong,last = hexp
                choidx = chosung2index(cho)
                if(choidx!=-1):
                    result.append(choidx)
                jungidx = jungsung2index(jung)
                if(jungidx!=-1):
                    result.append(jungidx)
                jongidx= jongsung2index(jong)
                if(jongidx!=-1):
                    result.append(jongidx)
                
            else:
                cho,jung,last = hexp
                choidx = chosung2index(cho)
                if(choidx!=-1):
                    result.append(choidx)
                jungidx = jungsung2index(jung)
                if(jungidx!=-1):
                    result.append(jungidx)
            #result.append(last)
        else:
            idx = -1
            for k,v in enumerate(etc_chars):
                if(v.lower()==i.lower()):
                    result.append(k+len(chosung)+len(jungsung)+len(jongsung))
            if(idx!=-1):
                result.append(idx)
    result = [i+4 for i in result]
    return result

def vector2text(vec):
    etc_chars = list(ENGS+special_chars)
    result = ""
    len_cho = len(chosung)
    len_jung = len(jungsung)
    len_jong = len(jongsung)
    len_etcchars = len(etc_chars)
    for i,v in enumerate(vec):
        if(v < 2 or v==3):
            continue
        if(v==2):
            break
        if(v<len_cho+4):#chosung
            result+=chosung[v-4]
        elif(v<len_cho+len_jung+4):#jungsung
            result += jungsung[(v-4-len_cho)]
        elif(v<len_cho+len_jung+len_jong+4):#jongsung
            result += jongsung[(v-len_cho-len_jung-4)]
        elif(v<len_cho+len_jung+len_jong+len_etcchars+4):
            result += etc_chars[v-len_cho-len_jung-len_jong-4]
    return result

def moasseugi(inputtext):
    t1 = []
    for i in inputtext:
        t1.append(i)

    return hangeulJoin(t1)

In [79]:
import numpy as np
import os
from glob import glob
import json
from pathlib import Path
import torch
import torchaudio
from torchaudio import transforms as audioT
import sentencepiece as spm
import jamo

class DatasetLoader:
    def __init__(self,is_train_dataset:bool,batch_size,top_dataset_folder = "자유대화 음성(일반남녀)"):
        super().__init__()
        section_folder = "Training"
        if(is_train_dataset==False):
            section_folder = "Validation"
        top_folder = os.path.join(top_dataset_folder,section_folder)
        tmp_audios = glob(os.path.join(top_folder,"*\\*.wav"))
        audios = []
        for aud in tmp_audios:
            ffolder = os.path.dirname(aud)
            fname = os.path.basename(aud)
            fn = Path(fname).stem
            trans_path = os.path.join(ffolder,fn+".json")
            if(os.path.exists(trans_path)==True):
                trans = self.__getTranscription(trans_path)
                trans = sentence_filter(trans)
                trans = replaceBracket(trans)
                #print("transcript2:",transcription)
                #trans = text2vector(trans)
                #trans = replaceBracket(trans)
                if(len(trans)>0):
                    audios.append(aud)
        self.__audios = audios
        use_subset = not True
        spm_folder= os.path.dirname(top_dataset_folder)
        self.sp = spm.SentencePieceProcessor(model_file="spm_bpe_4096_second.model")
        if(is_train_dataset==True and use_subset==True):
            audio_len = len(audios)
            self.__audios = self.__audios[:audio_len//10]
        self.__audio_len = len(self.__audios)
        self._batch_size = batch_size
    def vector2text(self,vec):
        return self.sp.DecodeIds(vec)
    def __len__(self):
        return self.__audio_len//self._batch_size
    def __getTranscription(self,fpath):
        with open(fpath,"r",encoding="UTF-8") as fp:
            data = json.load(fp)["발화정보"]["stt"]
            #data = sentence_filter(data)
            #data = jamo.h2j(data)
            #data = jamo.j2hcj(data)
        return data
    def __getRawAudioData(self,fpath,with_pytorch=True):
        if(with_pytorch==True):
            data,sample_rate = torchaudio.load(fpath)
            return data,sample_rate
        else:
            import librosa
            data,sample_rate = librosa.load(fpath)
            return data,sample_rate

    def __getAudioData(self,fpath,as_raw_data:bool = False,with_pytorch=True):
        data,sample_rate = self.__getRawAudioData(fpath,with_pytorch)
        if(as_raw_data==False):
            if(with_pytorch==True):
                resample_rate = 16000
                mfcc_transforms = audioT.MelSpectrogram(sample_rate=resample_rate,n_fft=400,n_mels=96,hop_length=160) #MFCC(sample_rate=resample_rate,n_mfcc=96)
                
                data = torchaudio.functional.resample(data,sample_rate,resample_rate)
                data = mfcc_transforms(data)
                #data_mean = data.mean(dim=-1).unsqueeze(-1)
                #data_std = data.std(dim=-1).unsqueeze(-1)
                #data =  (data-data_mean)/(data_std+1e-35)
                data = torch.nan_to_num(data)
            
                data = torch.nn.functional.pad(data,pad=(0,4))
                data = data.transpose(1,2)
                data = data.squeeze(0)
                length = data.size(0)-4

            else:
                import librosa
                data= librosa.feature.mfcc(y=data,sr=sample_rate).T
                data = torch.from_numpy(data)
                length = data.size(0)
        return data,length
        
        
    def shuffleDataset(self):
        random.shuffle(self.__audios)
    def __getitem__(self,idx):
        if(idx>=self.__len__()):
            raise StopIteration()
        st_idx,ed_idx = idx*self._batch_size,min((idx+1)*self._batch_size,self.__audio_len)
        audios = []
        audio_lengths = []
        raw_transcripts = []
        transcripts = []
        transcript_length = []
        for i in range(st_idx,ed_idx):
            fpath = self.__audios[i]
            fname = os.path.basename(fpath)
            #print(fname)
            ffolder = os.path.dirname(fpath)
            transcript_fpath = os.path.join(ffolder,Path(fname).stem+".json")
            transcription = self.__getTranscription(transcript_fpath)
            audio_data,data_length = self.__getAudioData(fpath)
            audios.append(audio_data)
            audio_lengths.append(data_length)
            ori_trans = transcription
            
            transcription = bracketContentFilter(transcription)
            
            #print("transcript:",transcription)
            transcription = sentence_filter(transcription)
            
            raw_transcripts.append((ori_trans,transcription,fname))
            transcription = replaceBracket(transcription)
            #print("transcript2:",transcription)
            
            #transcription = jamo.j2hcj(jamo.h2j(transcription))
            
            transcription =  [0] + text2vector(transcription) #self.sp.EncodeAsIds(transcription) #[1]+text2vector(transcription)+[2]
            #print("transcription:",transcription)
            transcripts.append(torch.from_numpy(np.array(transcription)))

            transcript_length.append(len(transcription))#temp length
        audios = torch.nn.utils.rnn.pad_sequence(audios,batch_first=True)
        audio_lengths = np.array(audio_lengths)
        transcript_length = np.array(transcript_length)
        #transcripts = np.array(transcripts)
        return audios,audio_lengths,transcripts,transcript_length#,raw_transcripts
        
if(__name__=="__main__"):
    debug_dataset =  False
    if(debug_dataset==True):
        from parasol import Composer
        text_composer = Composer()
        dataset_loader = DatasetLoader(is_train_dataset=True,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
        for i, (audios,audio_lengths,transcripts,transcript_length,raw_transcripts) in enumerate(dataset_loader):
            #print("audios:",len(audios),"transcripts:",len(transcripts))
            sample_audio = audios[0]
            sample_transcripts = transcripts[0]
            #print(sample_audio.shape)
            print("  ",sample_transcripts)
            decode = vector2text(sample_transcripts[1:].cpu().numpy().tolist())#dataset_loader.vector2text(sample_transcripts[1:].cpu().numpy().tolist())
            #decoded = jamo.j2hcj(jamo.h2j(decode))
            #decoded2 = text_composer.compose(decoded)
            print(f"  => '{decode}'")

            label = raw_transcripts[0]
            if(1==1):#label[0].find("(")!=-1):
                filtered_label = label[1]
                print(str(i).zfill(6),f"'{label[0]}'")
                print(f"\t\t'{label[1]}'")
                print(f"\t\t\t{label[2]}")
            if(i>10):
                break

            

In [6]:
if(__name__=="__main__"):
    traindataset_loader = DataLoader(15)
    from matplotlib import pyplot as plt
    index = 0
    for specs,specs_lengths,transcripts,transcript_lengths in tqdm(traindataset_loader):
        #print(specs.shape,specs_lengths.shape,transcripts.shape,transcript_lengths.shape)
        fig, ax = plt.subplots(nrows=2, sharex=True)
        mfcc= specs[0].T
        print("mfcc:",mfcc.shape)
        print("\t",(mfcc.min(),mfcc.max()))#(np.min(mfcc),np.max(mfcc)))
        img = librosa.display.specshow(librosa.power_to_db(mfcc, ref=np.max),
                                      x_axis='time', y_axis='mel',fmin=100, fmax=8000,
                                      ax=ax[0])
        fig.colorbar(img, ax=[ax[0]])
        ax[0].set(title='Mel spectrogram')
        ax[0].label_outer()
        img = librosa.display.specshow(mfcc, x_axis='time', ax=ax[1])
        fig.colorbar(img, ax=[ax[1]])
        ax[1].set(title='MFCC')
        plt.show()
        if(index>20):
            break
        index+=1


NameError: name 'DataLoader' is not defined

* https://pytorch.org/audio/stable/generated/torchaudio.models.emformer_rnnt_model.html?highlight=rnnt#torchaudio.models.emformer_rnnt_model

In [34]:
net = emformer_rnnt_model(
        input_dim=96,
        encoding_dim=1024,
        num_symbols=256,
        segment_length=16,
        right_context_length=4,
        time_reduction_input_dim=128,
        time_reduction_stride=4,
        transformer_num_heads=8,
        transformer_ffn_dim=2048,#1536,
        transformer_num_layers=20,
        transformer_dropout=0.1,
        transformer_activation="gelu",
        transformer_left_context_length=30,
        transformer_max_memory_size=0,
        transformer_weight_init_scale_strategy="depthwise",
        transformer_tanh_on_mem=True,
        symbol_embedding_dim=512,
        num_lstm_layers=3,
        lstm_layer_norm=True,
        lstm_layer_norm_epsilon=1e-3,
        lstm_dropout=0.3,
    )
device = torch.device("cuda:0" if(torch.cuda.is_available()==True) else "cpu")
net = net.to(device)
#net.device

* https://pytorch.org/audio/stable/generated/torchaudio.models.RNNTBeamSearch.html?highlight=beamsearch#torchaudio.models.RNNTBeamSearch

In [35]:
def predict(net,dataset):
    net.eval()
    state = None
    infer = torchaudio.models.RNNTBeamSearch(net,blank=0)
    print("device:",device)
    with torch.no_grad():
        for (features,feat_lens,labels,label_lens) in dataset:
            feat_lens,label_lens = torch.from_numpy(feat_lens),torch.from_numpy(label_lens)

            features = nn.utils.rnn.pad_sequence(features,batch_first=True)
            labels = nn.utils.rnn.pad_sequence(labels,batch_first=True)
            
            features = features.to(device)
            labels = labels.to(device)
            feat_lens = feat_lens.to(device)
            label_lens = label_lens.to(device)
            
            output = infer(features.float(),feat_lens,10)
            tmp_output = output[0]
            
            ori_decode = vector2text(labels[0][1:].cpu().numpy().tolist())#dataset.vector2text(labels[0][1:].cpu().numpy().tolist())
            decode = vector2text(tmp_output[0][1:])#dataset.vector2text(tmp_output[0][1:])
            #decoded = jamo.j2hcj(jamo.h2j(decode))
            #decoded2 = text_composer.compose(decoded)
            
            print("-"*10)
            print("\torigin:",ori_decode)#vector2text(labels[0].cpu().numpy().tolist()))
            print("\tresult:",decode)#vector2text(tmp_output[0]))
            print("\toriginal vec:",labels[0])
            #with_visible_tag
            print("\traw_output:",tmp_output[0])
            
            """
            ori_string = dataset.token2String(labels.cpu().numpy()[0].tolist())
            pred_string = dataset.token2String(output[0][0])
            print("\tori string: ",ori_string)
            print("\toutput result:",output[0][0])
            print("\topt string: ",pred_string)
            ori_string = dataset.token2String(labels.cpu().numpy()[0].tolist(),with_visible_tag=False)
            pred_string = dataset.token2String(output[0][0],with_visible_tag=False)
            print(f"\topt' string: '{pred_string}'")
            print("\twer:",wer(pred_string,ori_string))
            print("\tcer:",cer(pred_string,ori_string))
            """
            del features,feat_lens,labels,label_lens
            torch.cuda.empty_cache()
  

In [ ]:
del features,feat_lens,labels,label_lens
torch.cuda.empty_cache()

In [78]:
initial_learning_rate = 0.00008
optimizer = optim.Adam(net.parameters(),lr=initial_learning_rate)#,rho=0.95,eps=1e-8,weight_decay=0.0)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.95) #.CosineAnnealingLR(optimizer,T_max=100,eta_min=0.001)

In [22]:
weights = torch.load("./rnnt4r2.pth")
net.load_state_dict(weights)

<All keys matched successfully>

In [80]:
traindataset2 = DatasetLoader(is_train_dataset=True,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")


In [12]:
predict(net,traindataset2)#testdataset)

device: cuda:0


C:\Users\imscs\AppData\Local\Programs\Python\Python37\lib\site-packages\torchaudio\functional\functional.py:572: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


----------
	origin: ㄱㅡㄹㅓㄴ ㅅㅔㅁㅇㅣㅈㅣ ㄱㅡ ㅇㅣㅎㅜ ㅊㅓㄴㅎㅘㅇㅇㅣ ㅈㅣㄴㅏㄷㅏㄱㅏ ㄴㅐㄹㅕㄱㅇㅡㄹ ㄷㅡㄷㄱㅗ ㅇㅣㄹㅡㅁㅇㅡㄹ ㄱㅡㅁㅅㅗㄹㅗㄹㅗ ㅂㅏㄲㅝㅆㄷㅐ
	result: ㄱㅡㄴㄷㅔ ㅇㅙ
	original vec: tensor([ 0,  4, 41,  9, 27, 48, 98, 13, 28, 60, 15, 43, 16, 43, 98,  4, 41, 98,
        15, 43, 22, 36, 98, 18, 27, 48, 22, 32, 65, 15, 43, 98, 16, 43,  6, 23,
         7, 23,  4, 23, 98,  6, 24,  9, 29, 45, 15, 41, 52, 98,  7, 41, 51,  4,
        31, 98, 15, 43,  9, 41, 60, 15, 41, 52, 98,  4, 41, 60, 13, 31,  9, 31,
         9, 31, 98, 11, 23,  5, 37, 64,  7, 24], device='cuda:0',
       dtype=torch.int32)
	raw_output: [0, 4, 41, 48, 7, 28, 98, 15, 33]
----------
	origin: ㄱㅡㄹㅓㅎㄱㅜㄴㅏ ㄱㅡ ㅅㅐㅇㄱㅏㄱㅇㅡㄹ ㅁㅗㅅ ㅎㅐㅆㅇㅓ
	result: ㄱㅡㄴㄷㅔ ㄴㅏ
	original vec: tensor([ 0,  4, 41,  9, 27, 71,  4, 36,  6, 23, 98,  4, 41, 98, 13, 24, 65,  4,
        23, 45, 15, 41, 52, 98, 10, 31, 63, 98, 22, 24, 64, 15, 27],
       device='cuda:0', dtype=torch.int32)
	raw_output: [0, 4, 41, 48, 7, 28, 98, 6, 23]
----------
	origin: ㅋㅓㄷㅏㄹㅏㄴ ㅈㅓㄴㅊㅓㄹㅇㅡㄹ ㅌㅏㄱㅗ ㅇㅓㄷㅣㄹㅗㄴㄱㅏ ㄱㅏㄱㅗ ㅇㅣㅆㄴㅡㄴㄷㅔ ㅊㅏㅇㅂㅏㄲㅇㅡㄹㅗ ㅎㅐㅅㅅ

----------
	origin: ㄴㅏㄴ ㅎㅗㅇㅋㅗㅇㅇㅔㅅㅓ ㅇㅔㄱㅡㅌㅏㄹㅡㅌㅡㄹㅡㄹ ㅁㅓㄱㅇㅓㅂㅘㅆㄴㅡㄴㄷㅔ ㅍㅗㄹㅡㅌㅜㄱㅏㄹㅎㅏㄱㅗㄴ ㅈㅗㅁ ㄷㅏㄹㅡㄴㄱㅏ
	result: ㄱㅡㄴㄷㅔ ㄴㅏ
	original vec: tensor([ 0,  6, 23, 48, 98, 22, 31, 65, 19, 31, 65, 15, 28, 13, 27, 98, 15, 28,
         4, 41, 20, 23,  9, 41, 20, 41,  9, 41, 52, 98, 10, 27, 45, 15, 27, 11,
        32, 64,  6, 41, 48,  7, 28, 98, 21, 31,  9, 41, 20, 36,  4, 23, 52, 22,
        23,  4, 31, 48, 98, 16, 31, 60, 98,  7, 23,  9, 41, 48,  4, 23],
       device='cuda:0', dtype=torch.int32)
	raw_output: [0, 4, 41, 48, 7, 28, 98, 6, 23]


KeyboardInterrupt: 

In [81]:
traindataset = DatasetLoader(is_train_dataset=True,batch_size=2,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
testdataset = DatasetLoader(is_train_dataset=False,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
    

In [ ]:
if(__name__=="__main__"):
    #initial_learning_rate = 0.0001
    #import nemo.collections.asr as nemo_asr

    max_num_epoch = 100
    #optimizer = optim.Adam(net.parameters(),lr=initial_learning_rate)#,rho=0.95,eps=1e-8,weight_decay=0.0)
    criterion = torchaudio.transforms.RNNTLoss(blank=-1,reduction='sum',clamp=1.0)#,clamp=1.0)#,fused_log_softmax=True)#,#,clamp=1.0)#'sum')
    #criterion = nemo_asr.losses.rnnt.RNNTLoss(num_classes=0)
    #traindataset = DatasetLoader(is_train_dataset=True,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
    #testdataset = DatasetLoader(is_train_dataset=False,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
    #torch.save(net.parameters(),"rnnt.pth")
    #predict(net,testdataset)
    
    for epoch in range(max_num_epoch):
        index = 0
        traindataset.shuffleDataset()
        losses = []
        net.train()
        for (features,feat_lens,labels,label_lens) in tqdm((traindataset)):
            feat_lens,label_lens = torch.from_numpy(feat_lens),torch.from_numpy(label_lens)
            if(len(features)==0):
                continue
            features = nn.utils.rnn.pad_sequence(features,batch_first=True)#.unsqueeze(1).transpose(2, 3)
            labels = nn.utils.rnn.pad_sequence(labels,batch_first=True)
            
            """
            sorted_lens,indices = torch.sort(feat_lens.view(-1),dim=0,descending=True)
            features = features[indices]
            labels = labels[indices]
            feat_lens = sorted_lens
            label_lens = label_lens[indices]
            """
            
            #features = features.float()
            
            features = features.to(device)
            labels = labels.to(device)
            feat_lens = feat_lens.to(device)
            label_lens = label_lens.to(device)
            
            
            optimizer.zero_grad()

            joint_outputs,src_length,tgt_length,*_ = net(features.float(),feat_lens,labels,label_lens)
            
            outputs = joint_outputs#
            #outputs = nn.functional.log_softmax(joint_outputs,dim=-1)

            #loss = criterion(log_probs=outputs,targets=(labels[...,1:]).int(),input_lengths=(src_length-1).int() ,target_lengths=(label_lens-1).int())
            loss = criterion(outputs,(labels[...,1:]).contiguous().int(),(src_length-0).int() ,(label_lens-1).int())

            
            loss.backward()
            optimizer.step()
            del features,feat_lens,labels,label_lens
            torch.cuda.empty_cache()
            losses.append(loss.item())
            if(index%20==0 and index>0):
                torch.save(net.state_dict(),"r.pth")
            if(index%100==0):
                print(loss.item())
            index+=1
        print(f"ep={epoch}] loss= ",np.mean(losses))
        #scheduler.step()
    torch.save(net.state_dict(),"rnnt4r2.pth")
    predict(net,testdataset2)

  0%|                                                                              | 1/19965 [00:00<2:56:30,  1.89it/s]

346.698486328125


  1%|▍                                                                           | 101/19965 [00:28<1:47:50,  3.07it/s]

318.6271667480469


  1%|▋                                                                           | 183/19965 [00:50<1:39:51,  3.30it/s]

In [76]:
torch.save(net.state_dict(),"rnnt4r2.pth")

In [77]:
predict(net,traindataset2)

device: cuda:0
----------
	origin: ㄱㅡㄹㅓㄴ ㅅㅔㅁㅇㅣㅈㅣ ㄱㅡ ㅇㅣㅎㅜ ㅊㅓㄴㅎㅘㅇㅇㅣ ㅈㅣㄴㅏㄷㅏㄱㅏ ㄴㅐㄹㅕㄱㅇㅡㄹ ㄷㅡㄷㄱㅗ ㅇㅣㄹㅡㅁㅇㅡㄹ ㄱㅡㅁㅅㅗㄹㅗㄹㅗ ㅂㅏㄲㅝㅆㄷㅐ
	result: ㄱㅡㄹㅓㅎㅈㅣ ㅋㅡㄹㅔㅍㅔㄴㅡㄴ ㅇㅝㄴㄹㅐ ㅍㅡㄹㅏㅇㅅㅡ ㄷㅣㅈㅓㅌㅡㅇㅣㄴㄷㅔ ㅇㅛㅈㅡㅁㅇㅡㄴ ㅇㅣㄹㅂㅗㄴ ㅋㅡㄹㅔㅍㅔㄷㅗ ㅇㅠㅁㅕㅇㅎㅏㄷㅓㄹㅏ
	original vec: tensor([ 0,  4, 41,  9, 27, 48, 98, 13, 28, 60, 15, 43, 16, 43, 98,  4, 41, 98,
        15, 43, 22, 36, 98, 18, 27, 48, 22, 32, 65, 15, 43, 98, 16, 43,  6, 23,
         7, 23,  4, 23, 98,  6, 24,  9, 29, 45, 15, 41, 52, 98,  7, 41, 51,  4,
        31, 98, 15, 43,  9, 41, 60, 15, 41, 52, 98,  4, 41, 60, 13, 31,  9, 31,
         9, 31, 98, 11, 23,  5, 37, 64,  7, 24], device='cuda:0',
       dtype=torch.int32)
	raw_output: [0, 4, 41, 9, 27, 71, 16, 43, 98, 19, 41, 9, 28, 21, 28, 6, 41, 48, 98, 15, 37, 48, 9, 24, 98, 21, 41, 9, 23, 65, 13, 41, 98, 7, 43, 16, 27, 20, 41, 15, 43, 48, 7, 28, 98, 15, 35, 16, 41, 60, 15, 41, 48, 98, 15, 43, 52, 11, 31, 48, 98, 19, 41, 9, 28, 21, 28, 7, 31, 98, 15, 40, 10, 29, 65, 22, 23, 7, 27, 9, 23]
----------
	origin: ㄱㅡㄹㅓㅎㄱㅜㄴㅏ ㄱㅡ ㅅㅐㅇㄱㅏㄱㅇㅡ

KeyboardInterrupt: 